# Ask and Shine - Recommender System

**The aim of our recommender system is to recommend a category that the person to be gifted most likely also likes based on the purchases other consumers have done previosly on amazon**

For our recommender we used a **collaborative filtering** for **implicit data** based on the work of Hu et al.:

**[Hu et al (2008), Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf)**


The following Python implementation of the approach was used by Ben Frederickson:

[Faster Implicit Matrix Factorization](http://www.benfrederickson.com/fast-implicit-matrix-factorization/)

And the following implementation examples were used as guidance:

* [ALS Implicit Collaborative Filtering](https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe)
* [A Gentle Introduction to Recommender Systems with Implicit Feedback](https://jessesw.com/Rec-System/)
* [Intro to Implicit Matrix Factorization: Classic ALS with Sketchfab Models](https://www.ethanrosenthal.com/2016/10/19/implicit-mf-part-1/)

In [8]:
#imports 
import pandas as pd 
import numpy as np
import sys
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
import implicit
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from sklearn import metrics
from itertools import combinations 

In [9]:
conda install -c conda-forge implicit

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


# Load and Preprocess Data

In [10]:
#load csv file
df_amazon_reviews = pd.read_csv('final_dataset.csv', dtype=str)
df_amazon_reviews.head()

,Unnamed: 0,overall,reviewerID,asin,reviewText,summary,category
0,0,4.0,A2C9XE9I8RSKNX,0887248845,Nice and snazzy to put on letters to a sponsor...,I'm pleased,Arts_Crafts_and_Sewing
1,1,5.0,A1WGU1NUVWJPR0,0988174979,This book has so many quilting possibilities! ...,LOVE IT! Blocks are fun and easy to make!,Arts_Crafts_and_Sewing
2,2,5.0,A333SV82Y549LK,0988174979,"What can I say, I LOVE this book of farm-relat...",Adorable and Easy to do Quilt Designs,Arts_Crafts_and_Sewing
3,3,5.0,A39EMFHHUTUX8M,0988174979,This is a great book and will keep a quilter q...,This is a great book and will keep a quilter q...,Arts_Crafts_and_Sewing
4,4,4.0,A27WH4VW6VS7GC,0988174979,Good,Four Stars,Arts_Crafts_and_Sewing


In [11]:
#rename reviewerID to reviewer
df_amazon_reviews = df_amazon_reviews.rename({'reviewerID': 'reviewer'}, axis=1) 

#create list with customers and count the products they have bought for each category = purchases
df_grouped = df_amazon_reviews.groupby(['category','reviewer']).size().reset_index(name="purchases")
print("Restructered Data Set\n", df_grouped.head())
print("Length of Data Set", len(df_grouped))
print("Max Value of Purchases", df_grouped['purchases'].max())

df_grouped.head()

Restructered Data Set
                  category              reviewer  purchases
0  Arts_Crafts_and_Sewing  A0116143FGG14B3OZ7UG          4
1  Arts_Crafts_and_Sewing  A0228801ER3IBSLNOT5W          3
2  Arts_Crafts_and_Sewing  A0272662PGC9D57A0G5C          7
3  Arts_Crafts_and_Sewing  A03033388K5LWDWFS9R9          6
4  Arts_Crafts_and_Sewing  A0323662LGDA74F87HNI          9
Length of Data Set 2251715
Max Value of Purchases 1478


,category,reviewer,purchases
0,Arts_Crafts_and_Sewing,A0116143FGG14B3OZ7UG,4
1,Arts_Crafts_and_Sewing,A0228801ER3IBSLNOT5W,3
2,Arts_Crafts_and_Sewing,A0272662PGC9D57A0G5C,7
3,Arts_Crafts_and_Sewing,A03033388K5LWDWFS9R9,6
4,Arts_Crafts_and_Sewing,A0323662LGDA74F87HNI,9


In [12]:
#Filter outliers
q_low = df_grouped["purchases"].quantile(0.001)
q_hi  = df_grouped["purchases"].quantile(0.999)

df_grouped = df_grouped[(df_grouped["purchases"] < q_hi) & (df_grouped["purchases"] > q_low)]
print("Restructered Data Set\n", df_grouped.head())
print("Length of Data Set", len(df_grouped))
print("Max Value of Purchases", df_grouped['purchases'].max())

Restructered Data Set
                  category              reviewer  purchases
0  Arts_Crafts_and_Sewing  A0116143FGG14B3OZ7UG          4
1  Arts_Crafts_and_Sewing  A0228801ER3IBSLNOT5W          3
2  Arts_Crafts_and_Sewing  A0272662PGC9D57A0G5C          7
3  Arts_Crafts_and_Sewing  A03033388K5LWDWFS9R9          6
4  Arts_Crafts_and_Sewing  A0323662LGDA74F87HNI          9
Length of Data Set 1435381
Max Value of Purchases 35


In [13]:
#Get Sparsity
n_reviewer = df_grouped.reviewer.unique().shape[0]
n_categories = df_grouped.category.unique().shape[0]

print('Number of reviewer: {}'.format(n_reviewer))
print('Number of categories: {}'.format(n_categories))
print('Sparsity: {:4.3f}%'.format(float(df_grouped.shape[0]) / float(n_reviewer*n_categories) * 100))

Number of reviewer: 970929
Number of categories: 19
Sparsity: 7.781%


In [ ]:
# Create a numeric reviewer_id and category_id column
df_grouped['reviewer'] = df_grouped['reviewer'].astype("category")
df_grouped['category'] = df_grouped['category'].astype("category")
df_grouped['reviewer_id'] = df_grouped['reviewer'].cat.codes
df_grouped['category_id'] = df_grouped['category'].cat.codes

# The implicit library expects data as a item-user matrix two matricies are created, 
#one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((df_grouped['purchases'].astype(float), (df_grouped['category_id'].astype(np.int32), df_grouped['reviewer_id'].astype(np.int32))))
sparse_user_item = sparse.csr_matrix((df_grouped['purchases'].astype(float), (df_grouped['reviewer_id'].astype(np.int32), df_grouped['category_id'].astype(np.int32))))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=32, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)

# Evaluate the Model 

In [ ]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [ ]:
product_train, product_test, product_users_altered = make_train(sparse_user_item, pct_test = 0.2)

In [ ]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)  

def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [ ]:
#first create test data
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(model.user_factors), sparse.csr_matrix(model.item_factors.T)], product_test)

In [ ]:
#Just for trying out the recommender
# Find the 3 most similar to a specific category
item_id = 13
n_similar = 3

# Get the user and item vectors from our trained model
user_vecs = model.user_factors
item_vecs = model.item_factors

# Calculate the vector norms
item_norms = np.sqrt((item_vecs * item_vecs).sum(axis=1))

# Calculate the similarity score, grab the top N items and
# create a list of item-score tuples of most similar artists
scores = item_vecs.dot(item_vecs[item_id]) / item_norms
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
similar = sorted(zip(top_idx, scores[top_idx] / item_norms[item_id]), key=lambda x: -x[1])

# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print (df_grouped.category.loc[df_grouped.category_id == idx].iloc[0])

# Built the LookUp Table 

In [14]:
categegory_list = df_grouped.category.unique()
combination_list = list(combinations(categegory_list, 2))

In [15]:
print(combination_list)

[('Arts_Crafts_and_Sewing', 'Automotive'), ('Arts_Crafts_and_Sewing', 'CDs_and_Vinyl'), ('Arts_Crafts_and_Sewing', 'Cell_Phones_and_Accessories'), ('Arts_Crafts_and_Sewing', 'Clothing_Shoes_and_Jewelry'), ('Arts_Crafts_and_Sewing', 'Electronics'), ('Arts_Crafts_and_Sewing', 'Grocery_and_Gourmet_Food'), ('Arts_Crafts_and_Sewing', 'Home_and_Kitchen'), ('Arts_Crafts_and_Sewing', 'Industrial_and_Scientific'), ('Arts_Crafts_and_Sewing', 'Kindle_Store'), ('Arts_Crafts_and_Sewing', 'Luxury_Beauty'), ('Arts_Crafts_and_Sewing', 'Movies_and_TV'), ('Arts_Crafts_and_Sewing', 'Musical_Instruments'), ('Arts_Crafts_and_Sewing', 'Patio_Lawn_and_Garden'), ('Arts_Crafts_and_Sewing', 'Prime_Pantry'), ('Arts_Crafts_and_Sewing', 'Sports_and_Outdoors'), ('Arts_Crafts_and_Sewing', 'Tools_and_Home_Improvement'), ('Arts_Crafts_and_Sewing', 'Toys_and_Games'), ('Arts_Crafts_and_Sewing', 'Video_Games'), ('Automotive', 'CDs_and_Vinyl'), ('Automotive', 'Cell_Phones_and_Accessories'), ('Automotive', 'Clothing_Shoes_

In [16]:
df_grouped_added = df_grouped;

In [17]:
def getCatById(id):
    return df_grouped_added.category.loc[df_grouped_added.category_id == id].iloc[0]
def getUserById(id):
    return df_grouped_added.reviewer.loc[df_grouped_added.reviewer_id == id].iloc[0]
def getIdByCat(cat):
    return df_grouped_added.category_id.loc[df_grouped_added.category == cat].iloc[0]
def getIdByUser(reviewer):
    return df_grouped_added.reviewer_id.loc[df_grouped_added.reviewer == reviewer].iloc[0]

In [18]:
#add a fake reviewer to the dataset
lookup_table_2 = combination_list
print("Number of Combinations ", len(combination_list))
for idx, combi in enumerate(combination_list):
    
    df_grouped_added = df_grouped;
    
    #add fake reviewer to the dataset
    df_grouped_added = df_grouped_added.append({'category': combi[0], 'reviewer': 'combination'+str(idx+1), 'purchases': 4}, ignore_index=True)
    df_grouped_added = df_grouped_added.append({'category': combi[1], 'reviewer': 'combination'+str(idx+1), 'purchases': 4}, ignore_index=True)
    
    # Create a numeric reviewer_id and category_id column
    df_grouped_added['reviewer'] = df_grouped_added['reviewer'].astype("category")
    df_grouped_added['category'] = df_grouped_added['category'].astype("category")
    df_grouped_added['reviewer_id'] = df_grouped_added['reviewer'].cat.codes
    df_grouped_added['category_id'] = df_grouped_added['category'].cat.codes
    
    # The implicit library expects data as a item-user matrix two matricies are created, 
    #one for fitting the model (item-user) 
    # and one for recommendations (user-item)
    sparse_item_user = sparse.csr_matrix((df_grouped_added['purchases'].astype(float), (df_grouped_added['category_id'].astype(np.int32), df_grouped_added['reviewer_id'].astype(np.int32))))
    sparse_user_item = sparse.csr_matrix((df_grouped_added['purchases'].astype(float), (df_grouped_added['reviewer_id'].astype(np.int32), df_grouped_added['category_id'].astype(np.int32))))
    
    # Initialize the als model and fit it using the sparse item-user matrix
    model = implicit.als.AlternatingLeastSquares(factors=32, regularization=0.1, iterations=20)
    
    # Calculate the confidence by multiplying it by our alpha value.
    alpha_val = 15
    data_conf = (sparse_item_user * alpha_val).astype('double')
    
    #Fit the model
    model.fit(data_conf)
    
    #Recommend Category
    print("For the categories:", combi[0], combi[1])
    
    # Use the implicit recommender:
    recommended = model.recommend(getIdByUser('combination'+str(idx+1)), sparse_user_item)
    
    print("This category is recommended: ", getCatById(recommended[0][0]))
    
    lookup_table_2[idx] = lookup_table_2[idx] + (getCatById(recommended[0][0]), )

Number of Combinations  171



For the categories: Arts_Crafts_and_Sewing Automotive
This category is recommended:  Grocery_and_Gourmet_Food



For the categories: Arts_Crafts_and_Sewing CDs_and_Vinyl
This category is recommended:  Automotive



For the categories: Arts_Crafts_and_Sewing Cell_Phones_and_Accessories
This category is recommended:  Kindle_Store



For the categories: Arts_Crafts_and_Sewing Clothing_Shoes_and_Jewelry
This category is recommended:  Automotive


...


For the categories: Sports_and_Outdoors Toys_and_Games
This category is recommended:  Patio_Lawn_and_Garden



For the categories: Sports_and_Outdoors Video_Games
This category is recommended:  Grocery_and_Gourmet_Food



For the categories: Tools_and_Home_Improvement Toys_and_Games
This category is recommended:  Grocery_and_Gourmet_Food



For the categories: Tools_and_Home_Improvement Video_Games
This category is recommended:  Grocery_and_Gourmet_Food



For the categories: Toys_and_Games Video_Games
This category is recommended:  Tools_and_Home_Improvement


In [23]:
#create dataframe of lockup table and export it
df_lookUp = pd.DataFrame(list(lookup_table_2), columns=['category1','category2', 'recommended'])
df_lookUp.head()
df_lookUp.to_csv('lookUp_table.csv', index=False)